In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import numpy as np
import librosa
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/faysond/music_genre_model3.json"
SCALER_PATH = "/content/drive/MyDrive/faysond/scaler3.npy"
LABEL_ENCODER_PATH = "/content/drive/MyDrive/faysond/label_encoder3.npy"

model = xgb.Booster()
model.load_model(MODEL_PATH)

In [ ]:
scaler = np.load(SCALER_PATH, allow_pickle=True).item()
label_encoder = np.load(LABEL_ENCODER_PATH, allow_pickle=True)

In [ ]:

def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=30)  

    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr).T, axis=0)

    return np.hstack([mfccs, chroma, mel, contrast, tonnetz])


In [ ]:

test_files = ["/content/drive/MyDrive/faysond/Stephane.wav", "/content/drive/MyDrive/faysond/classical7.wav", "/content/drive/MyDrive/faysond/hiphop4.wav"]

for file in test_files:
    if not os.path.exists(file):
        print(f" File not found: {file}")
        continue

    features = extract_features(file)

    features = features.reshape(1, -1)

    features_scaled = scaler.transform(features)

    dmatrix = xgb.DMatrix(features_scaled)

    prediction = model.predict(dmatrix)

    predicted_index = int(np.argmax(prediction)) if len(prediction.shape) > 1 else int(prediction[0])

    predicted_genre = label_encoder[predicted_index]

    print(f"{file}: {predicted_genre}")


/content/drive/MyDrive/faysond/Stephane.wav: hiphop
/content/drive/MyDrive/faysond/classical7.wav: classical
/content/drive/MyDrive/faysond/hiphop4.wav: hiphop
